# Convolutional Neural Network를 활용한 Sentiment Analysis

이번 튜토리얼은 앞선 Word2Vec 튜토리얼에서 활용한 IMDB 데이터셋을 활용하여 Sentiment Analysis를 하고자 합니다. 내용은 모델 구현을 제외한 Recurrent Neural Network와 크게 다르지 않습니다.

In [1]:
import pandas as pd
from IPython.display import display
from tqdm import tqdm
import random
import numpy as np

import tensorflow as tf

train = pd.read_csv("../src/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

/Users/taekyoon/Envs/tensorflow_pr/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def make_multi_class(labels):
    output = np.zeros([labels.shape[0], 2])
    for i, (label, o) in enumerate(zip(labels, output)):
        if label == 0:
            output[i,0] = 1
        else:
            output[i,1] = 1
    
    return output

In [3]:
labels = make_multi_class(train['sentiment'])

In [4]:
train['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

## Normalizing Text

In [5]:
from bs4 import BeautifulSoup

def normalize_text(text):
    return str.lower(BeautifulSoup(text, 'html.parser').get_text().strip())

In [6]:
normalize_text(train['review'][0])

'"with all this stuff going down at the moment with mj i\'ve started listening to his music, watching the odd documentary here and there, watched the wiz and watched moonwalker again. maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. some of it has subtle messages about mj\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring. some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him.the actual feature film bit when it finally starts is only on for 2

## Tokenization 

nltk 라이브러리를 활용하여 단어를 tokenize 해줍니다. 
활용하는 함수는 nltk.word_tokenize() 입니다.

In [7]:
import nltk
def word_tokenize(text):
    return [token.replace("''", '"').replace("``", '"') for token in nltk.tokenize.word_tokenize(text)]

In [8]:
word_tokenize(normalize_text(train['review'][0]))

['"',
 'with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 "'ve",
 'started',
 'listening',
 'to',
 'his',
 'music',
 ',',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 ',',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again',
 '.',
 'maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent',
 '.',
 'moonwalker',
 'is',
 'part',
 'biography',
 ',',
 'part',
 'feature',
 'film',
 'which',
 'i',
 'remember',
 'going',
 'to',
 'see',
 'at',
 'the',
 'cinema',
 'when',
 'it',
 'was',
 'originally',
 'released',
 '.',
 'some',
 'of',
 'it',
 'has',
 'subtle',
 'messages',
 'about',
 'mj',
 "'s",
 'feeling',
 'towards',
 'the',
 'press',
 'and',
 'also',
 'th

## Data Packaging

학습을 위해 input과 target 데이터를 하나의 인스턴스로 패키징 합니다.

In [9]:
def preprocess_data(data, labels):
    prepro_data = []
    for item, label in tqdm(zip(data, labels)):
        instance = {
                    "input_seq": word_tokenize(normalize_text(item)),
                    "label": label
                    }
        prepro_data.append(instance)
    return prepro_data

In [10]:
dataset = preprocess_data(train['review'], labels)

25000it [01:00, 410.98it/s]


In [11]:
dataset[0]

{'input_seq': ['"',
  'with',
  'all',
  'this',
  'stuff',
  'going',
  'down',
  'at',
  'the',
  'moment',
  'with',
  'mj',
  'i',
  "'ve",
  'started',
  'listening',
  'to',
  'his',
  'music',
  ',',
  'watching',
  'the',
  'odd',
  'documentary',
  'here',
  'and',
  'there',
  ',',
  'watched',
  'the',
  'wiz',
  'and',
  'watched',
  'moonwalker',
  'again',
  '.',
  'maybe',
  'i',
  'just',
  'want',
  'to',
  'get',
  'a',
  'certain',
  'insight',
  'into',
  'this',
  'guy',
  'who',
  'i',
  'thought',
  'was',
  'really',
  'cool',
  'in',
  'the',
  'eighties',
  'just',
  'to',
  'maybe',
  'make',
  'up',
  'my',
  'mind',
  'whether',
  'he',
  'is',
  'guilty',
  'or',
  'innocent',
  '.',
  'moonwalker',
  'is',
  'part',
  'biography',
  ',',
  'part',
  'feature',
  'film',
  'which',
  'i',
  'remember',
  'going',
  'to',
  'see',
  'at',
  'the',
  'cinema',
  'when',
  'it',
  'was',
  'originally',
  'released',
  '.',
  'some',
  'of',
  'it',
  'has',


## Creat Vocabulary

NLP에서는 일반적으로 Word Embedding을 위해 모델에 lookup embedding 모듈을 둡니다. 따라서 모델에 들어가는 문장 sequence 내용은 전부 인덱스번호 sequence로 들어갑니다. 이들을 단어에서 인덱스로 또는 인덱스에서 단어로 변환해주기 위해 Vocabulary를 만듭니다.

In [12]:
__PAD__ = "@@PAD@@"
__UNK__ = "@@UNK@@"

def create_vocabulary(dataset):
    word2idx, idx2word = {__PAD__: 0, __UNK__: 1}, {0: __PAD__, 1: __UNK__}
    indexer = 2
    
    for instance in tqdm(dataset):
        for token in instance["input_seq"]:
            if token not in word2idx:
                word2idx[token] = indexer
                idx2word[indexer] = token
                indexer += 1
    return word2idx, idx2word

In [13]:
word2idx, idx2word = create_vocabulary(dataset)

100%|██████████| 25000/25000 [00:00<00:00, 27305.93it/s]


In [14]:
display("Vocab Size: " + str(len(word2idx)))

'Vocab Size: 143352'

## Data Shuffle 및 Test Dataset 구분

In [15]:
from random import shuffle

dev_portion = 0.1
split_pos = int(len(dataset) * (1 - dev_portion))

shuffle(dataset)

train_dataset, dev_dataset = dataset[:split_pos], dataset[split_pos:]

In [16]:
display("Train Dataset Count: " + str(len(train_dataset)))
display("Dev Dataset Count: " + str(len(dev_dataset)))

'Train Dataset Count: 22500'

'Dev Dataset Count: 2500'

## Model 생성

CNN모델을 생성하기 위해 Model class를 구현합니다.

In [17]:
class SentimentModel(object):
    def __init__(self, 
                 _vocab_size, 
                 _hidden_dim, 
                 _num_classes, 
                 _seq_length,
                 _num_layers=1,
                 _filter_sizes=[2, 3, 4],
                 _learning_rate=0.01):
        
        self.vocab_size = _vocab_size
        self.hidden_dim = _hidden_dim
        self.num_layers = _num_layers
        self.num_classes = _num_classes
        self.filter_sizes = _filter_sizes
        self.seq_length = _seq_length
        
        self.learning_rate = _learning_rate
        
        self.inputs = tf.placeholder(tf.int64, [None, None])
        self.targets = tf.placeholder(tf.int64, [None, self.num_classes])
        
        self.weight_initializer = tf.contrib.layers.xavier_initializer()
        self.const_initializer = tf.constant_initializer(0.0)
        self.emb_initializer = tf.random_uniform_initializer(minval=-1.0, maxval=1.0)
        
        self.train_op = None
        self.cost = None
        
        self.build_model()
        
    def _word_embedding(self, inputs, reuse=False):
        with tf.variable_scope('word_embedding', reuse=reuse):
            weight = tf.get_variable('w', [self.vocab_size, self.hidden_dim], initializer=self.emb_initializer)
            word_vectors = tf.nn.embedding_lookup(weight, inputs, name='word_vector')  # (N, T, M) or (N, M)
            return word_vectors
        
    def _conv_layer(self, inputs):
        pooled_outputs = []
        for i, filter_size in enumerate(self.filter_sizes):
            with tf.variable_scope('conv_maxpool_%d' % (i+1)):
                filter_shape = [filter_size, self.hidden_dim, 1, self.hidden_dim]
                w = tf.get_variable('w', shape=filter_shape, initializer=self.weight_initializer)
                b = tf.get_variable('b', shape=[self.hidden_dim], initializer=self.const_initializer)
                
                conv = tf.nn.conv2d(inputs, w, strides=[1, 1, 1, 1], padding='VALID') + b
                relu = tf.nn.relu(conv)
                pooled = tf.nn.max_pool(relu, [1, self.seq_length - filter_size + 1, 1, 1], [1, 1, 1, 1], padding='VALID')
                pooled_outputs.append(pooled)
        
        pooled = tf.concat(pooled_outputs, 3)
        pooled = tf.reshape(pooled, [-1, pooled.shape[-1]])
        
        return pooled
    
    def build_model(self):
        word_vectors = self._word_embedding(self.inputs)
        word_vectors = tf.expand_dims(word_vectors, -1)
        encoded_vector = self._conv_layer(word_vectors)
        
        logits = tf.layers.dense(encoded_vector, self.num_classes)
        
        self.outputs = tf.argmax(logits, 1)
        self.cost, self.train_op = self._build_ops(logits, self.targets)
        
    def _build_ops(self, logits, targets):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=targets))
        train_op = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(cost)
        
        return cost, train_op
    
    def train(self, session, _inputs, _targets):
        return session.run([self.train_op, self.cost], 
                           feed_dict={self.inputs: _inputs, 
                                      self.targets: _targets})
    
    def test(self, session, _inputs, _targets):
        prediction_check = tf.equal(self.outputs, tf.argmax(self.targets, 1))
        accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))
        
        return session.run([self.targets, self.outputs, accuracy],
                           feed_dict={self.inputs:_inputs,
                                      self.targets:_targets})    

## Batch Loader 구현

Batch Loader는 학습할 데이터를 구성하는 역할을 합니다. 따라서, 딥러닝 학습을 하는데 중요한 역할을 합니다. 또한, NLP에서는 모델학습 속도를 높히는데 중요한 역할을 하기도 합니다. 주로 Bucketing 기법을 Batch Loader에 적용하여 모델학습 속도를 높힙니다.

In [18]:
class BatchLoader(object):
    def __init__(self, dataset, word2idx, limit_passage_len=None, shuffle=True, batch_size=30):
        self.dataset = dataset
        self.word2idx = word2idx
        self.batch_index_pool = []

        self.max_passage_len = -1
        self.limit_passage_len = limit_passage_len
        self.batch_size = batch_size
        self.current_position = 0
        self.shuffle = shuffle

        if self.limit_passage_len:
            self.passage_len_limit()
        
        self.bins = self.build_bins(self.dataset)
        self.max_passage_len = self.get_max_passage_len()
        self.reset()
        print("Total number of data: ", len(self.dataset))
        print("===================================")
        print("")

    def get_total_instance(self):
        return len(self.dataset)
    
    def get_max_passage_len(self):
        max_passage_len = -1
        if self.max_passage_len == -1:
            for instance in self.dataset:
                passage_len = len(instance["input_seq"])
                if passage_len > max_passage_len:
                    max_passage_len = passage_len
        else:
            max_passage_len = self.max_passage_len
        return max_passage_len
    
    def set_max_passage_len(self, max_passage_len):
        self.max_passage_len = max_passage_len
    
    def build_bins(self, data):
        round_to_power = lambda x: int(x / 100 + 1) * 100

        doc_len = map(lambda x: round_to_power(len(x["input_seq"])), data)

        bins = {}
        for i, l in enumerate(doc_len):
            if l not in bins:
                bins[l] = []
            bins[l].append(i)
        return bins

    def reset(self):
        self.current_position = 0

        if self.shuffle:
            for ixs in iter(self.bins.values()):
                random.shuffle(ixs)

        self.batch_index_pool = []

        for l, ixs in self.bins.items():
            m = len(ixs)
            k = m / self.batch_size if m % self.batch_size == 0 else m / self.batch_size + 1
            k = int(k)
            ixs_list = [(ixs[self.batch_size * i:min(m, self.batch_size * (i + 1))], l) for i in range(k)]
            self.batch_index_pool += ixs_list

        # randomly shuffle the mini-batches
        if self.shuffle:
            random.shuffle(self.batch_index_pool)

    def passage_len_limit(self):
        filtered_dataset = []
        for instance in tqdm(self.dataset):
            if len(instance["input_seq"]) <= self.limit_passage_len:
                filtered_dataset.append(instance)
        print("Dataset filtered ", len(filtered_dataset), "/", len(self.dataset))
        print("Dataset Changed!")
        self.dataset = filtered_dataset

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def next(self):
        if self.current_position == len(self.batch_index_pool):
            print("#" * 30)
            print("End Iteration")
            self.reset()
            raise StopIteration()

        indicies = self.batch_index_pool[self.current_position][0]
        self.current_position += 1
        
        batch_size = len(indicies)

        passage = np.zeros((batch_size, self.max_passage_len), dtype='int64')
        labels = []

        for i, idx in enumerate(indicies):
            instance = self.dataset[idx]
            passage_input = [self.word2idx[word] for word in instance["input_seq"]]

            passage[i, :len(passage_input)] = np.array(passage_input)
            labels.append(instance["label"])
        
        target = np.array(labels, dtype='float32')
        batch_dict = {"input_seq": passage,
                      "target": target}

        return batch_dict

In [19]:
class AverageMeter(object):
    """
    Computes and stores the average and current value
    Borrowed from ImageNet training in PyTorch project
    https://github.com/pytorch/examples/tree/master/imagenet
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Batch Loader 준비

일반 컴퓨터에서는 긴 문장에 대한 classification이 어렵기 때문에 문장의 길이를 100으로 제한하여 학습합니다.

In [20]:
train_loader = BatchLoader(train_dataset, word2idx, limit_passage_len=100)
dev_loader = BatchLoader(dev_dataset, word2idx, limit_passage_len=100)
dev_loader.set_max_passage_len(train_loader.get_max_passage_len())

100%|██████████| 2500/2500 [00:00<00:00, 725306.77it/s]

Dataset filtered  2079 / 22500
Dataset Changed!
Total number of data:  2079

Dataset filtered  219 / 2500
Dataset Changed!
Total number of data:  219



## Model 실험

In [21]:
vocab_size = len(word2idx)
hidden_dim = 100
num_classes = 2
max_seq_len = train_loader.get_max_passage_len()
print("max_seq_len: " + str(max_seq_len))

tf.reset_default_graph()
model = SentimentModel(vocab_size, 
                       hidden_dim, 
                       num_classes,
                       max_seq_len)

max_seq_len: 100


## Model 저장 준비

모델을 어느정도 학습하고나면 저장 할 필요가 있다. 여기서는 tf.train.Saver를 활용하여 모델의 파라메터들을 저장할 것이다.

In [27]:
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state('./model/cnn')

In [32]:
num_epochs = 3
checkpoint = 10

with tf.Session() as sess:
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        avg_loss = AverageMeter()
        avg_acc = AverageMeter()
        for i, data in enumerate(tqdm(train_loader)):
            inputs, targets = data["input_seq"], data["target"]
            _, loss = model.train(sess, inputs, targets)
            _, _, accuracy = model.test(sess, inputs, targets)
            
            avg_loss.update(loss)
            avg_acc.update(accuracy)
            
            if i % checkpoint == 0:
                message = "epoch: %d, step: %d, loss: %1.2f, accuracy: %1.2f" % (epoch+1, i+1, avg_loss.avg, avg_acc.avg)
                print(message)
    
    dev_avg_acc = AverageMeter()
    for i, data in enumerate(tqdm(dev_loader)):
        inputs, targets = data["input_seq"], data["target"]
        _, _, accuracy = model.test(sess, inputs, targets)
        avg_acc.update(accuracy)
        
    message = "Dev set accuracy: %1.2f" % (avg_acc.avg)
    print(message)
    saver.save(sess, './model/cnn/cnn_sentiment.ckpt')


0it [00:00, ?it/s]
Exception in thread Thread-21:
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/taekyoon/Envs/tensorflow_pr/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/taekyoon/Envs/tensorflow_pr/bin/../lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

1it [00:00,  3.38it/s]

epoch: 1, step: 1, loss: 0.68, accuracy: 0.57


11it [00:02,  3.79it/s]

epoch: 1, step: 11, loss: 1.06, accuracy: 0.71


21it [00:05,  3.74it/s]

...

epoch: 3, step: 61, loss: 0.11, accuracy: 0.99


70it [00:20,  3.44it/s]
1it [00:00,  7.52it/s]

##############################
End Iteration


9it [00:01,  7.08it/s]


##############################
End Iteration
Dev set accuracy: 0.97
